# Data preparation 

Install and import the necessary libraries.

In [1]:
!pip install smplx
!pip install git+https://github.com/nghorbani/configer
!pip install trimesh
!pip install pyrender

     |████████████████████████████████| 51kB 3.3MB/s 
  Cloning https://github.com/nghorbani/configer to /tmp/pip-req-build-oyjgn0ca
  Running command git clone -q https://github.com/nghorbani/configer /tmp/pip-req-build-oyjgn0ca
  Created wheel for configer: filename=configer-1.4.1-cp37-none-any.whl size=7848 sha256=0250d8fc0c59e16e0a1a435b613e771c724663b06ba283b17716bd53884a1006
  Stored in directory: /tmp/pip-ephem-wheel-cache-ucpbqtua/wheels/4a/5d/ef/47b4fb4b0c90f8049bf1fe0d52cc9d89a36b90465561a3c015
Successfully built configer
     |████████████████████████████████| 634kB 7.4MB/s 
     |████████████████████████████████| 1.2MB 7.7MB/s 
     |████████████████████████████████| 890kB 20.2MB/s 
     |████████████████████████████████| 1.2MB 31.3MB/s 
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-cp37-none-any.whl size=1745211 sha256=54bc050012e3f2f8ed4bd24353261c07b3a924db0f2964f3a13417d182575fb5
  Stored in directory: /root/.cache/pip/wheels/6c/00/7f/1dd736f380848720ad79a1a1de5

In [2]:
!git clone https://github.com/bronwynbiro/human_body_prior

Cloning into 'human_body_prior'...
remote: Enumerating objects: 901, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (179/179), done.
remote: Total 901 (delta 100), reused 101 (delta 20), pack-reused 699
Receiving objects: 100% (901/901), 25.68 MiB | 28.65 MiB/s, done.
Resolving deltas: 100% (519/519), done.


In [4]:
import pandas as pd
import numpy as np
import random

In [5]:
%cd human_body_prior/human_body_prior/

/content/human_body_prior/human_body_prior


We create a new file from the experiment data. There are features for the experiment number and frame number, and for each frame in each experiment we provide the values for the 72 SMPL body pose parameters. 

The code can be generated [here](https://github.com/bronwynbiro/human_body_prior/blob/master/generate_csv.py), or you can download the pkl.csv file from [here](https://drive.google.com/file/d/1KsapE49zsnLTI-gwj7rEUBgJ871Lx6Xl/view?usp=sharing) (recommended). Place it in the `human_body_prior/data` file. 

In [6]:
df = pd.read_csv('data/pkl.csv')  
df.head()

,experiment_num,frame_num,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72
0,0,0,-2.866536,0.075659,-1.145443,0.050882,-0.070009,0.026480,-0.074817,-0.001138,-0.017932,0.196515,0.049918,-0.024497,0.109902,-0.142839,-0.052636,0.351383,-0.082448,0.018162,0.057499,0.005253,-0.021140,-0.232680,0.123407,0.017125,-0.186094,-0.062421,-0.007275,0.077147,0.049321,-0.007620,-0.055069,0.068586,0.030873,-0.018599,0.033485,0.073937,0.083949,0.170684,0.005117,0.022195,0.061079,-0.381387,-0.016794,0.129674,0.406932,0.281262,-0.006526,-0.015036,0.146671,-0.161312,-0.896686,0.213676,0.438120,0.810122,0.327218,-1.556438,0.502480,0.385893,1.373885,-0.511340,0.054412,-0.121351,0.130947,-0.063638,0.153678,-0.198115,-0.033524,0.009195,-0.369428,-0.047161,-0.019448,0.321557
1,0,1,-2.838369,0.075735,-1.143715,0.040607,-0.067597,0.050557,-0.025319,-0.007044,-0.014076,0.192631,0.055741,-0.026506,0.117011,-0.140747,-0.054009,0.252279,-0.072263,0.007663,0.061674,0.001663,-0.025815,-0.238407,0.126472,0.011188,-0.193134,-0.060448,-0.014639,0.077598,0.051748,-0.007666,-0.055225,0.067399,0.033278,-0.019427,0.031957,0.077390,0.070754,0.190760,-0.014085,0.019257,0.068265,-0.385795,-0.013156,0.116901,0.414061,0.293120,0.011190,-0.022356,0.146521,-0.162157,-0.899592,0.216117,0.450869,0.819911,0.328563,-1.553030,0.505692,0.405574,1.333898,-0.486168,0.063756,-0.115439,0.134983,-0.059764,0.156251,-0.198110,-0.031229,0.004888,-0.363507,-0.047416,-0.019419,0.314796
2,0,2,-2.829223,0.090393,-1.148443,0.063496,-0.055999,0.065752,0.001324,-0.010209,-0.013241,0.173072,0.059233,-0.026757,0.064246,-0.153684,-0.042731,0.178812,-0.060165,-0.002129,0.061603,0.002669,-0.027728,-0.227414,0.133802,0.005039,-0.189296,-0.065465,-0.012650,0.090368,0.055797,-0.005831,-0.053037,0.064830,0.036626,-0.014236,0.031690,0.078841,0.124344,0.191155,-0.034091,0.020000,0.087399,-0.348454,-0.009559,0.099641,0.391766,0.329253,0.007764,-0.022938,0.147610,-0.174609,-0.893082,0.219475,0.465768,0.821067,0.340594,-1.593601,0.498695,0.445730,1.374859,-0.460087,0.067147,-0.116501,0.143336,-0.075582,0.153389,-0.195452,-0.027182,0.011999,-0.359782,-0.046835,-0.026848,0.310718
3,0,3,-2.825207,0.089673,-1.140104,0.072179,-0.063327,0.063198,0.019392,-0.015909,-0.011003,0.173403,0.065946,-0.026625,0.056366,-0.167516,-0.040978,0.181117,-0.056387,-0.000715,0.054548,0.009960,-0.028651,-0.231065,0.132051,0.002744,-0.201887,-0.065286,-0.011699,0.086380,0.064775,-0.005147,-0.061064,0.068311,0.038010,-0.023431,0.033477,0.076317,0.078042,0.198403,-0.018582,0.012670,0.091686,-0.342287,-0.012891,0.107499,0.381302,0.297573,0.013360,-0.017678,0.147299,-0.170567,-0.915190,0.212589,0.471162,0.831528,0.306601,-1.636701,0.534682,0.420950,1.412081,-0.505871,0.065309,-0.116460,0.141962,-0.075909,0.154490,-0.199943,-0.025886,0.007023,-0.359902,-0.045624,-0.022944,0.309714
4,0,4,-2.877287,0.085534,-1.024322,0.117452,-0.058068,0.036590,-0.000754,-0.010078,-0.016400,0.164260,0.048183,-0.020332,0.043498,-0.181379,-0.042301,0.230787,-0.052373,-0.000683,0.051952,0.005621,-0.027468,-0.235382,0.136976,0.006767,-0.204834,-0.076237,0.000580,0.089965,0.053683,-0.008654,-0.084683,0.073989,0.035373,-0.043408,0.047037,0.064760,0.072398,0.217221,-0.014488,0.015751,0.096391,-0.343589,-0.017429,0.098780,0.373383,0.291016,0.015239,-0.035457,0.154262,-0.165363,-0.920355,0.233473,0.463017,0.845733,0.365914,-1.572845,0.481966,0.462682,1.424765,-0.483427,0.092077,-0.111556,0.139014,-0.055702,0.147208,-0.200182,-0.039764,0.007090,-0.350670,-0.052173,-0.019537,0.300950


As in SMPL-X [[1](https://github.com/nghorbani/human_body_prior), [2](https://arxiv.org/abs/1904.05866)], we use do not use the pelvis or hand effexors, leaving us with 21 joints to consider. 

In [7]:
# Reshape into (n, 3, 21), ie ignore the pelvis and hand effexors
def reshape_vibe(arr):
  arr = arr.reshape(-1, 3, 24)
  n = arr.shape[0]
  reshaped = np.zeros(shape=(n, 3, 21))
  for i in range(n):
    reshaped[i] = arr[i, :, 1:22]
  
  return reshaped

To generate the training and test datasets, select 10% from each experiment as start sequences for our test sequences. 

In [8]:
def generate_datasets_for_training(data, timesteps_input, timesteps_output, sample_rate=0.1):
  n = len(data) 
  X_train = []
  X_test = []

  Y_train = []
  Y_test = []

  for experiment_num in range(145):
    # For each experiment, we generate sample_rate % to use as for testing
    n = len(df.loc[df['experiment_num'] == experiment_num])
    num_samples = int(sample_rate * n)
    to_test = random.sample(range(0, n-timesteps_input), num_samples)

    for i in range(0, n- timesteps_input-timesteps_output+1):
      # Add start of sequence indices to test arrays
      if i in to_test:
        X_test.append(i)
        Y_test.append(i)
      
      else:
        X_train.append(data[i:i+timesteps_input][::-1])
        Y_train.append(data[i+timesteps_input:i+timesteps_input+timesteps_output])

  # Convert all into np arrays
  X_train = np.asarray(X_train, dtype=float)
  X_test = np.asarray(X_test, dtype=float)

  Y_train = np.asarray(Y_train, dtype=float)
  Y_test = np.asarray(Y_test, dtype=float)

  return (X_train.shape[2], X_train, Y_train, X_test, Y_test)

In [9]:
timesteps_input, timesteps_output = 6, 6 
feats, X, Y, X_test, Y_test = generate_datasets_for_training(df, timesteps_input, timesteps_output)
print(feats, X.shape, X_test.shape)

74 (56860, 6, 74) (6362,)


# LSTM VAE 

We develop a Long Short-Term Memory (LSTM) Variational Autoencoder to forecast future poses. It learns a latent representation of human pose and regularizes the distribution of the latent code to be a normal distribution. We can then predict future poses given an input sequence. We used the following tutorials and code samples as reference:

[Keras tutorial](https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html)

[Seq2Seq tutorial](https://machinelearningmastery.com/define-encoder-decoder-sequence-sequence-model-neural-machine-translation-keras/)

[Github](https://github.com/keras-team/keras-io/blob/master/examples/nlp/lstm_seq2seq.py)

[Stack overflow](https://stackoverflow.com/questions/63987125/keras-lstm-vae-variational-autoencoder-for-time-series-anamoly-detection)

In [11]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras import backend as K

from keras import optimizers

n_sample = X.shape[0]
latent_dim = 32
features = X.shape[2]

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='loss', min_delta=0.001, patience=5, verbose=0, mode='auto',
    baseline=None, restore_best_weights=True)

def sampling(args):
    z_mean, z_log_sigma = args
    batch_size = tf.shape(z_mean)[0] 
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0., stddev=1.)
    return z_mean + K.exp(0.5 * z_log_sigma) * epsilon

encoder_inputs = Input(shape=(None, features))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

#z_layer
z_mean = Dense(latent_dim)(encoder_outputs)
z_log_sigma = Dense(latent_dim)(encoder_outputs)
z = Lambda(sampling)([z_mean, z_log_sigma])
repeated_context = RepeatVector(6)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, features))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = TimeDistributed(Dense(features, activation="linear")) #_x_decoded_mean
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

def vae_loss2(input_x, decoder1, z_log_sigma, z_mean):
    """ Calculate loss = reconstruction loss + KL loss for each data in minibatch """
    recon = K.mean(K.square(decoder1 - input_x)) * timesteps_input
    kl = -0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
    return recon + (0.0001 * kl)


# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.add_loss(vae_loss2(encoder_inputs, decoder_inputs, z_log_sigma, z_mean))

optimizer = optimizers.Adam()#clipvalue=0.5, lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.001)
model.compile(optimizer=optimizer, metrics=["mean_squared_error"])

model.fit([X, Y], Y, epochs=200, callbacks=[early_stop])

Epoch 1/200
1777/1777 [==============================] - 11s 5ms/step - loss: 42.1858 - mean_squared_error: 109.9443
Epoch 2/200
1777/1777 [==============================] - 9s 5ms/step - loss: 49.9157 - mean_squared_error: 111.1624
Epoch 3/200
1777/1777 [==============================] - 9s 5ms/step - loss: 51.3721 - mean_squared_error: 111.9581
Epoch 4/200
1777/1777 [==============================] - 9s 5ms/step - loss: 46.0788 - mean_squared_error: 105.9060
Epoch 5/200
1777/1777 [==============================] - 9s 5ms/step - loss: 43.2570 - mean_squared_error: 112.5325
Epoch 6/200
1777/1777 [==============================] - 9s 5ms/step - loss: 50.6767 - mean_squared_error: 115.6782
Epoch 7/200
1777/1777 [==============================] - 9s 5ms/step - loss: 45.7288 - mean_squared_error: 112.2591
Epoch 8/200
1777/1777 [==============================] - 9s 5ms/step - loss: 45.0562 - mean_squared_error: 112.6768
Epoch 9/200
1777/1777 [==============================] - 9s 5ms/step - 

In [12]:
encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = Input(shape=(latent_dim,), name="input_3")
decoder_state_input_c = Input(shape=(latent_dim,), name="input_4")
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

The code below generates an output sequence for the input sequence.

In [13]:
def decode_sequence(input_seq, timesteps_input, timesteps_output):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, features))

    # Start with the last frame of the input as the target
    target_seq[0,0,:] = input_seq[0,timesteps_input-1,:]

    decoded_pose = []
    
    for i in range(timesteps_input):
        output_pose, h, c = decoder_model.predict([target_seq] + states_value)
        decoded_pose.append(output_pose)

        # Update the target sequence (of length 1).
        target_seq[0,0,:] = output_pose

        # Update states
        states_value = [h, c]

    return np.array(decoded_pose)

The code below generates an output sequence for the input sequence, but generates multiple samples for each frame and chooses the best one. 

In [14]:
def decode_sequence_multisample(input_seq, timesteps_input, timesteps_output, num_samples):
    best_pose = None
    min_err = float("inf")
    for _ in range(num_samples):
      states_value = []

      # Get random sample from z
      for i in range(2):
        states_value.append(K.random_normal(shape=(1, latent_dim), mean=0., stddev=1.))

      # Generate empty target sequence of length
      target_seq = np.zeros((1, 1, features))
      target_seq[0,0,:] = input_seq[0,timesteps_input-1,:]

      decoded_pose = []
      
      # Get predicted sequence
      for i in range(timesteps_input):
          output_pose, h, c = decoder_model.predict([target_seq] + states_value)
          decoded_pose.append(output_pose)

          # Update the target sequence
          target_seq[0,0,:] = output_pose

          # Update states
          states_value = [h, c]

      decoded_pose = np.array(decoded_pose)
      joint_err_per_frame = []

      # Get the MPJPE for the sequence
      for time in range(timesteps_output):
        poses, predicted_poses = input_seq[0, time, :], decoded_pose[time]
        poses, predicted_poses = reshape_vibe(poses[2:]), reshape_vibe(predicted_poses[0, 0, 2:])
        for dim in range(poses.shape[1]): # 3 dimensions (x, y, z)
          for joint in range(poses.shape[-1]): # 21 joints
            frame_err = abs(poses[0, dim, joint] - predicted_poses[0, dim, joint])
            joint_err_per_frame.append(frame_err)
      
      # Return the best sequence
      err = sum(joint_err_per_frame) / len(joint_err_per_frame)
      if err < min_err:
        min_err = err
        best_pose = decoded_pose
      
    return best_pose

# Visualization

Note: Must be run on GPU, otherwise you may get the error "Invalid device ID".

In [15]:
# Select a random sample to visualize
idx, start = 139, 180
test_seq = df.loc[df['experiment_num'] == idx]

test_seq = test_seq[start_time:start_time+timesteps_input]
test_seq = test_seq.to_numpy()
print(test_seq.shape)

(6, 74)


In [ ]:
%cd ../

Run the visualization on the test sequence. You may need to adjust the expr_dir and bm_path. 

In [17]:
from human_body_prior.tools.visualization_tools import render_smpl_params
from notebooks.notebook_tools import show_image
from human_body_prior.tools.visualization_tools import imagearray2file
from human_body_prior.body_model.body_model import BodyModel
from human_body_prior.tools.model_loader import load_vposer
import torch
import numpy as np
import random

import os
os.environ['PYOPENGL_PLATFORM'] = 'egl'

expr_dir = '../vposer_v1_0'
bm_path =  'smpl/models/neutral.pkl' 

bm = BodyModel(bm_path=bm_path, batch_size=1, use_posedirs=True)
vp, ps = load_vposer(expr_dir, vp_model='snapshot')


for i in range(timesteps_input):
  sample = reshape_vibe(test_seq[i, 2:]).reshape(-1, 63)
  images = render_smpl_params(bm, sample).reshape(1,1,1,400,400,3)
  img = imagearray2file(images, "test_original.png")
  for j in range(len(img)):
    show_image(np.array(img)[j])

FileNotFoundError: ignored

Run the visualization on the predicted sequence, using the best of 5 samples. 

In [ ]:
test_seq = df[start:start+timesteps_input].to_numpy()
pred_seq = test_seq 
num_samples = 5
pred_seq = pred_seq.reshape((1,) + pred_seq.shape)
prediction = decode_sequence_multisample(pred_seq, timesteps_input, timesteps_output, num_samples)

for i in range(len(prediction)):
  predicted_body = prediction[i]
  predicted_body = reshape_vibe(predicted_body[0,0,2:]).reshape(-1, 63)
  images = render_smpl_params(bm, predicted_body).reshape(1,1,1,400,400,3)
  img = imagearray2file(images, "test_out.png")
  for j in range(len(img)):
    show_image(np.array(img)[j])

# Evaluation

We use Mean Per Joint Position Error (MPJPE) in metres as our evaluation metric, which measures the Euclidean distance between our generated future poses and the ground truth future poses. 

In [ ]:
import random
from collections import defaultdict

joint_err_per_frame = []

print("Running on {} sequences.".format(str(len(X_test))))
for start in X_test:
  # Get ground truth
  start = int(start)
  samples = df.iloc[start:start+timesteps_input].to_numpy()

  for time in range(timesteps_output):
    sample = samples
    sample = sample.reshape((1,) + sample.shape)
    decoded_pose = decode_sequence(sample, timesteps_input, timesteps_output)
    
    # To test using the best pose with multiple samples, uncomment below 
    #decoded_pose = decode_sequence_multisample(sample, timesteps_input, timesteps_output, 10)
    poses, predicted_poses = sample[0,i,:], decoded_pose[time, 0, 0, 2:]
    poses = reshape_vibe(poses[2:])
    predicted_poses = reshape_vibe(predicted_poses)
    for dim in range(poses.shape[1]): # 3 dimensions (x, y, z)
      for joint in range(poses.shape[-1]): # 21 joints
        frame_err = abs(poses[0, dim, joint] - predicted_poses[0, dim, joint])
        joint_err_per_frame.append(frame_err)

avg_frame = sum(joint_err_per_frame) / len(joint_err_per_frame)
print("Joint error per frame:", avg_frame)